In [ ]:
!pip install requests beautifulsoup4

  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]


In [27]:
import requests

url = "https://chp.co.il/autocompletion/product_extended"
params = {
    "term": "טחינה",  # Product name or partial name
    "from": 0,        # Pagination offset
    "u": "0.5238520595295993",  # Session or user ID (can be dynamic)
    "shopping_address": "מעלה אדומים",  # Your shopping address
    "shopping_address_city_id": 3616,  # City ID
    "shopping_address_street_id": 9000,  # Street ID
}

response = requests.get(url, params=params)
data = response.json()

# Print the first 3 product suggestions
for product in data[:5]:
    print(f"Product: {product['label']}")
    # print(f"ID: {product['id']}")
    print(f"Price Range: {product['parts'].get('price_range', 'N/A')}")
    print("-" * 45)


Product: טחינה אמיתית, 400 גרם
Price Range: ['9.90', '15.50']
---------------------------------------------
Product: טחינה גולמית, 500 גרם
Price Range: ['18.90', '20.90']
---------------------------------------------
Product: טחינה ירושלים, 500 גרם
Price Range: ['14.90', '18.50']
---------------------------------------------
Product: טחינה גולמית מעודנת, 310 גרם
Price Range: ['11.90', '14.30']
---------------------------------------------
Product: טחינה גולמית משומשום טבעי, 500 גרם
Price Range: ['7.90', '13.90']
---------------------------------------------


In [31]:
import requests
from bs4 import BeautifulSoup

url = "https://chp.co.il/main_page/compare_results"
params = {
    "shopping_address": "מעלה אדומים",
    "shopping_address_street_id": "9000",
    "shopping_address_city_id": "3616",
    "product_name_or_barcode": "טחינה",
    "product_barcode": "7290027600007_7290000112220",
    "from": "0",
    "num_results": "20",
}

resp = requests.get(url, params=params)
soup = BeautifulSoup(resp.text, "html.parser")

stores = []
headers = []
for row in soup.select("table tr"):
    cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
    if not cells:
        continue

    # capture headers for context
    if cells[0] == "רשת":
        headers = cells
        continue

    # skip duplicate address-only rows
    if len(cells) == 1:
        continue

    # build record according to current headers
    record = dict(zip(headers, cells))
    stores.append(record)

for s in stores:
    print(s)


{}
{'רשת': 'רמי לוי', 'שם החנות': 'מישור אדומים', 'כתובת החנות': 'איתם 1, מישור אדומים, מעלה אדומים', 'מבצע': '5.80 *', 'מחיר': '15.90'}
{'רשת': 'יוחננוף', 'שם החנות': 'מישור אדומים', 'כתובת החנות': 'חרובית 38, מישור אדומים, מעלה אדומים', 'מבצע': '5.90 *', 'מחיר': '16.00'}
{'רשת': 'שופרסל דיל', 'שם החנות': 'קניון מעלה אדומים', 'כתובת החנות': 'דרך קדם 5, מעלה אדומים', 'מבצע': '5.90 *', 'מחיר': '16.30'}
{'רשת': 'מעיין 2000', 'שם החנות': 'מעלה אדומים', 'כתובת החנות': 'פרי מגדים 19, מעלה אדומים', 'מבצע': '', 'מחיר': '15.90'}
{'רשת': 'מעיין 2000', 'שם החנות': 'קניון מעלה אדומים', 'כתובת החנות': 'הרכס 31, מעלה אדומים', 'מבצע': '', 'מחיר': '15.90'}
{'רשת': 'מעיין 2000', 'שם החנות': 'הצור', 'כתובת החנות': 'הצור 36, מעלה אדומים', 'מבצע': '', 'מחיר': '15.90'}
{'רשת': 'מעיין 2000', 'שם החנות': 'כיכר יהלום', 'כתובת החנות': 'כיכר יהלום 1, מעלה אדומים', 'מבצע': '', 'מחיר': '15.90'}
{'רשת': 'חצי חינם אונליין', 'שם החנות': 'חצי חינם אונליין', 'אתר אינטרנט': 'https://shop.hazi-hinam.co.il/', 'מבצע': '5

In [ ]:
def get_product_barcode():
    pass



In [ ]:
import requests
from bs4 import BeautifulSoup


def find_best_price(
        shopping_address="מעלה אדומים", 
        shopping_address_street_id="9000",
        shopping_address_city_id="3616",
        product_name_or_barcode="טחינה",
        product_barcode="7290027600007_7290000112220",
        fromm="0",
        num_results="20",
    ):
    
    url = "https://chp.co.il/main_page/compare_results"
    params = {
        "shopping_address": shopping_address,
        "shopping_address_street_id": shopping_address_street_id,
        "shopping_address_city_id": shopping_address_city_id,
        "product_name_or_barcode": product_name_or_barcode,
        "product_barcode": product_barcode,
        "from": fromm,
        "num_results": num_results,
    }

    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
        if not cells:
            continue

        # capture headers for context
        if cells[0] == "רשת":
            headers = cells
            continue

        # skip duplicate address-only rows
        if len(cells) == 1:
            continue

        # build record according to current headers
        record = dict(zip(headers, cells))
        stores.append(record)

    # for s in stores:
    #     print(s)
    
    return stores[1]


find_best_price()

{'רשת': 'רמי לוי',
 'שם החנות': 'מישור אדומים',
 'כתובת החנות': 'איתם 1, מישור אדומים, מעלה אדומים',
 'מבצע': '5.80 *',
 'מחיר': '15.90'}

In [ ]:
# TODO: 
# Add YAML support using pyyaml
# Add caching so you don’t hammer the site for every run.
# Handle missing products or multiple best stores.
# Expand to full optimization (e.g., minimize total cost across N supermarkets).

In [ ]:
# import yaml
from pathlib import Path

# === 1. Load grocery list ===
# with open(Path("groceries.yaml"), "r", encoding="utf-8") as f:
    # groceries = yaml.safe_load(f)["items"]

groceries = ["בצל", "חלב", "ביצים"]

print("Groceries to check:", groceries)

# === 2. Function to get best price & store (use your existing scraper) ===
# from chp_scraper import find_best_price  # <-- create/rename your scraping function

results = []
for item in groceries:
    print(f"\n🔎 Checking: {item}")
    
    best = find_best_price(item)  # e.g. returns {'store': 'רמי לוי', 'price': 15.9, 'address': '...'}
    results.append({"product": item, **best})

# === 3. Summary output ===
print("\n=== Recommended purchases ===")
for r in results:
    print(f"{r['product']}: {r['store']} — ₪{r['price']}  ({r['address']})")


Groceries to check: ['בצל', 'חלב', 'ביצים']

🔎 Checking: בצל

🔎 Checking: חלב

🔎 Checking: ביצים

=== Recommended purchases ===


KeyError: 'store'

In [16]:
import requests

def get_barcode(name: str) -> str | None:
    url = "https://chp.co.il/autocompletion/product_extended"
    params = {
        "term": name,                     # e.g. "טחינה בארכה"
        "from": 0,
        "u": "0.5238",                    # any float string
        "shopping_address": "מעלה אדומים",
        "shopping_address_city_id": 3616,
        "shopping_address_street_id": 9000,
    }

    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()

    if not data:
        print("No matches")
        return None

    # Filter for the exact product if needed
    for item in data:
        if "בארכה" in item["label"] and "500" in item["label"]:
            return item["id"]

    return data[0]["product_code"]

print(get_barcode("טחינה בארכה"))


7290027600007_7290000112220


In [ ]:
#!/usr/bin/env python3

# import yaml
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import argparse


FROM = 0
NUM_RESULTS = 20

ADDRESS_DICT = {
    "מעלה אדומים" : {"city_id": "3616", "street_id": "9000"}
}


def parse_arguments():
    ap = argparse.ArgumentParser(description="Find best grocery prices using CHP.")
    ap.add_argument("--input", "-i", default='../input/groceries.txt', # required=True,
                        help="Path to YAML/JSON/TXT file listing grocery items")
    ap.add_argument("--address", "-a", default="מעלה אדומים", # required=True,
                        help="Shopping city for the search")
    return ap.parse_args()


def _fetch_barcode(product_name):
    url = "https://chp.co.il/autocompletion/product_extended"
    params = {
        "term": product_name,                     # e.g. "טחינה בארכה"
        "from": 0,
        "u": "0.5238",                    # any float string
        "shopping_address": "מעלה אדומים",
        "shopping_address_city_id": 3616,
        "shopping_address_street_id": 9000,
    }

    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()

    if not data:
        print("No matches")
        return None

    # Filter for the exact product if needed
    for item in data:
        if product_name in item["label"]:
            return item["id"]

    return data[0]["id"]


def _scrape_price_table(
        shopping_address="מעלה אדומים", 
        shopping_address_street_id="9000",
        shopping_address_city_id="3616",
        product_name="טחינה",
        product_barcode="7290027600007_7290000112220",
        fromm=FROM, num_results=NUM_RESULTS
):
    url = "https://chp.co.il/main_page/compare_results"
    params = {
        "shopping_address": shopping_address,
        "shopping_address_street_id": shopping_address_street_id,
        "shopping_address_city_id": shopping_address_city_id,
        "product_name_or_barcode": product_name,
        "product_barcode": product_barcode,
        "from": fromm,
        "num_results": num_results,
    }

    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
        if not cells:
            continue

        # capture headers for context
        if cells[0] == "רשת":
            headers = cells
            continue

        # skip duplicate address-only rows
        if len(cells) == 1: continue

        # build record according to current headers
        record = dict(zip(headers, cells))
        stores.append(record)

    # for s in stores:
    #     print(s)
    
    return stores[1]


def find_best_price(shopping_address, street_id, 
                    city_id, product_name,):

    product_barcode = _fetch_barcode(product_name)
    if not product_barcode:
        return None
    
    return _scrape_price_table(shopping_address, street_id, city_id, 
                               product_name, product_barcode)


def get_address_details(address):
    if address not in ADDRESS_DICT.keys:
        return None
    
    res = ADDRESS_DICT[address]
    city_id = res['city_id']
    street_id = res['street_id']
    
    return street_id, city_id 


res = find_best_price(shopping_address, 
                street_id,
                ßcity_id,
                product_name)




def get_barcode(name: str, shopping_address: str) -> str | None:
    """Query CHP autocomplete API for the first matching product_code."""



def main():
    args = parse_arguments()

    # Load grocery list (YAML for flexibility)
    with open(args.input, "r", encoding="utf-8") as f:
        groceries = yaml.safe_load(f)

    print(groceries)

    for item in groceries:
        # Get barcode: 
        url = "https://chp.co.il/autocompletion/product_extended"
        params = {
            "term": name,
            "from": 0,
            "u": "0.5238",                # random float string is fine
            "shopping_address": 'מעלה אדומים',
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
        }
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        if not data:
            print(f"No matches for {name}")
            return None
        barcode = data[0]["product_code"]
        if barcode:
            print(f"{item}: {barcode}")


if __name__ == "__main__":
    main()



NameError: name 'shopping_address' is not defined

In [40]:
# Load grocery list .txt file
with open("input/groceries.txt", "r", encoding="utf-8") as f:
    groceries = [line.strip() for line in f]
print(groceries)

for item in groceries:
    # Get barcode: 
    url = "https://chp.co.il/autocompletion/product_extended"
    params = {
        "term": item,
        "from": 0,
        "u": "0.5238",  # random float string is fine
        "shopping_address": "מעלה אדומים",
        "shopping_address_city_id": 3616,
        "shopping_address_street_id": 9000,
    }
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    if not data:
        print(f"No matches for {item}")

    barcode = data[0]["id"]
    if barcode:
        print(f"{item}: {barcode}")

    print("[i] Moving on to finding the best price...")
    url = "https://chp.co.il/main_page/compare_results"
    params = {
        "shopping_address": "מעלה אדומים",
        "shopping_address_street_id": "9000",
        "shopping_address_city_id": "3616",
        "product_name_or_barcode": item,
        "product_barcode": barcode,
        "from": 0,
        "num_results": 20,
    }

    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
    
        if not cells:
            continue
        
        # capture headers for context
        if cells[0] == "רשת":
            headers = cells
            continue

        # skip duplicate address-only rows
        if len(cells) == 1: continue

        # build record according to current headers
        record = dict(zip(headers, cells))
        stores.append(record)

    # print(cells)

    print(stores[1])
    print("-" * 50)

    del headers, stores, cells, soup, resp



['גזר', 'כרוב לבן', 'בצל יבש']
גזר: 7290027600007_7290000000121
[i] Moving on to finding the best price...
{'רשת': 'רמי לוי', 'שם החנות': 'מישור אדומים', 'כתובת החנות': 'איתם 1, מישור אדומים, מעלה אדומים', 'מבצע': '', 'מחיר': '1.80'}
--------------------------------------------------
כרוב לבן: 7290027600007_7290000000138
[i] Moving on to finding the best price...
{'רשת': '\u200c\u200b\u200c\u200bר\u200c\u200c\u200b\u200c\u200dC\u200b\u200bb\u200b\u200b\u200b\u200bמ\u200b\u200b\u200c\u200cיQ\u200d\u200c\u200b \u200b\u200b\u200bל\u200b\u200d\u200bo\u200d\u200b\u200cו\u200c\u200cי\u200b\u200c\u200c\u200c\u200b\u200c7\u200b\u200b\u200c\u200by\u200c\u200dy\u200d\u200c\u200b\u200c\u200b\u200b\u200c', 'שם החנות': '\u200b\u200c\u200bמ\u200b\u200b\u200cי\u200c\u200c\u200d\u200c\u200c\u200c\u200d1\u200c\u200d\u200d\u200d\u200cn\u200b\u200b\u200cQ\u200c\u200bש\u200b\u200b\u200bו\u200b\u200d\u200c\u200b9\u200d\u200b\u200d\u200b\u200c\u200c\u200bk\u200b\u200b\u200b\u200b8\u200d\u200b\u200b\u200c\u2

In [ ]:
for item in groceries:
    # --- find barcode ---
    r = requests.get(
        "https://chp.co.il/autocompletion/product_extended",
        params={
            "term": item,
            "from": 0,
            "u": "0.5238",
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
        },
        timeout=10
    )
    r.raise_for_status()
    data = r.json()
    if not data:
        print(f"No matches for {item}")
        continue

    barcode = data[0]["id"]
    print(f"{item}: {barcode}")

    # --- price compare: use ONLY barcode ---
    resp = requests.get(
        "https://chp.co.il/main_page/compare_results",
        params={
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
            "product_barcode": barcode,
            "from": 0,
            "num_results": 20,
        },
        timeout=10
    )
    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
        if not cells: continue
        if cells[0] == "רשת":
            headers = cells
            continue
        if len(cells) == 1: continue
        stores.append(dict(zip(headers, cells)))

    if stores:
        print(stores[1])
    print("-" * 50)


גזר: 7290027600007_7290000000121
{'רשת': 'שופרסל דיל', 'שם החנות': 'קניון מעלה אדומים', 'כתובת החנות': 'דרך קדם 5, מעלה אדומים', 'מבצע': '', 'מחיר': '1.90'}
--------------------------------------------------
כרוב לבן: 7290027600007_7290000000138
{'רשת': '\u200b\u200b\u200b\u200b\u200b\u200b\u200cש\u200b\u200c\u200c\u200c\u200dK\u200c\u200ca\u200c\u200d\u200c\u200cו\u200d\u200d9\u200c\u200d\u200d\u200cQ\u200d\u200c\u200bפ\u200c\u200c\u200bר\u200b\u200c\u200c\u200cס\u200c\u200cP\u200c\u200c\u200b\u200bs\u200c\u200b\u200b\u200b\u200bל\u200c\u200c\u200b\u200c\u200b2\u200c\u200c\u200c\u200c\u200b\u200c\u200bד\u200c\u200bי\u200b\u200c\u200c\u200c\u200cF\u200c\u200b\u200b\u200b\u200bD\u200dל\u200b\u200b\u200b\u200c\u200b\u200c\u200dC\u200b\u200c\u200c\u200d', 'שם החנות': '\u200b\u200b\u200b\u200c\u200bB\u200c\u200b\u200cS\u200c\u200c\u200b\u200cק\u200b\u200b\u200d\u200bq\u200b\u200d\u200d\u200cJ\u200c\u200bנ\u200c\u200bי\u200c\u200c\u200c\u200c\u200b\u200c\u200c\u200c\u200cו\u200c\u200b4\u200

In [ ]:
def do_it_all(item):
    # --- find barcode ---
    r = requests.get(
        "https://chp.co.il/autocompletion/product_extended",
        params={
            "term": item,
            "from": 0,
            "u": "0.5238",
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
        },
        timeout=10
    )
    print("-" * 50)

    r.raise_for_status()
    data = r.json()
    if not data:
        print(f"No matches for {item}")
        print("-" * 50)
        return

    barcode = data[0]["id"]
    print(f"{item}: {barcode}")

    # --- price compare ---
    resp = requests.get(
        "https://chp.co.il/main_page/compare_results",
        params={
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
            "product_barcode": barcode,
            "from": 0,
            "num_results": 20,
        },
        timeout=10
    )
    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
        if not cells: continue
        if cells[0] == "רשת":
            headers = cells
            continue
        if len(cells) == 1: continue
        stores.append(dict(zip(headers, cells)))

    if stores:
        print(stores[1])
    # print("-" * 50)

In [ ]:
import requests
import time
import yaml
import os
import requests

from collections import defaultdict
from bs4 import BeautifulSoup

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "X-Requested-With": "XMLHttpRequest"
})

grocery_data = defaultdict(list)

def do_it_all(item):
    # --- find barcode ---
    r = session.get(
        "https://chp.co.il/autocompletion/product_extended",
        params={
            "term": item,
            "from": 0,
            "u": "0.5238",
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
        },
        timeout=10
    )
    r.raise_for_status()
    data = r.json()
    if not data:
        print(f"No matches for {item}")
        return

    # take only the real barcode after the underscore
    product_barcode = data[0]["id"]
    # item_name = data[0]["value"]
    # print(f"{item}: {barcode}")

    # --- price compare ---
    resp = session.get(
        "https://chp.co.il/main_page/compare_results",
        params={
            "shopping_address": "מעלה אדומים",
            "shopping_address_city_id": 3616,
            "shopping_address_street_id": 9000,
            # "product_name_or_barcode": item_name,
            "product_barcode": product_barcode,
            "from": 0,
            "num_results": 20,
        },
        timeout=10
    )

    soup = BeautifulSoup(resp.text, "html.parser")

    stores, headers = [], []
    for row in soup.select("table tr"):
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]
        if not cells:
            continue
        if cells[0] == "רשת":
            headers = cells
            continue
        if len(cells) == 1:
            continue
        stores.append(dict(zip(headers, cells)))

    if stores:
        best_store = stores[1]
      
        best_price = best_store["מחיר"] if best_store["מבצע"] == '' else best_store["מבצע"].strip('*').strip(' ')
        best_market = best_store["רשת"]

        # add to final dictionary
        grocery_data[best_market].append([item, best_price])

    # print("-" * 50)
    time.sleep(3)

# ----------- main loop -------------
with open("input/groceries.txt", "r", encoding="utf-8") as f:
    groceries = [line.strip() for line in f]

n = len(groceries)
for i, item in enumerate(groceries):
    print(f'[{i+1}/{n}] {item}')
    do_it_all(item)

# Save to YAML
# with open("grocery_list.yaml", "w", encoding="utf-8") as f:
#     yaml.dump(dict(grocery_data), f, allow_unicode=True, default_flow_style=False)

# print("Saved grocery list to grocery_list.yaml")

# with open("grocery_list.txt", "w", encoding="utf-8") as f:
#     for store, items in grocery_data.items():
#         f.write(f"{store}:\n")
#         for item in items:
#             f.write(f"- {item}\n")
#         f.write("\n")

# print("Saved grocery list to grocery_list.txt")

# --- Build message text with RTL mark ---
RTL = '\u200F'
message_lines = []
for store, items in grocery_data.items():
    message_lines.append(f"{RTL}{store}:")
    for item, price in items:
        message_lines.append(f"{RTL}- {item} ({price})")
message_text = "\n".join(message_lines)

# --- Telegram bot credentials ---
bot_token = os.getenv("BOT_TOKEN")
chat_id = os.getenv("CHAT_ID")

# --- Send message ---
url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
payload = {
    "chat_id": chat_id,
    "text": message_text
}

response = requests.post(url, data=payload)
if response.ok:
    print("Message sent successfully!")
else:
    print("Error sending message:", response.text)
    

[1/4] כרוב לבן
[2/4] גזר
[3/4] בצל יבש
[4/4] טחינה בארכה
Message sent successfully!


In [8]:
# Prepend RTL mark to store names and items
def rtl_dict(data):
    new_data = {}
    for store, items in data.items():
        new_store = '\u200F' + store
        # convert each [item, price] to string with RTL mark
        new_items = ['\u200F' + f"{i[0]} ({i[1]})" if isinstance(i, list) else str(i) for i in items]
        new_data[new_store] = new_items
    return new_data

rtl_data = rtl_dict(grocery_data)

# Save to YAML
with open("grocery_list_rtl.yaml", "w", encoding="utf-8") as f:
    yaml.dump(rtl_data, f, allow_unicode=True, default_flow_style=False)

print("Saved grocery_list_rtl.yaml (with RTL marks)")

Saved grocery_list_rtl.yaml (with RTL marks)


In [ ]:
import os
import requests
from collections import defaultdict

# --- Your grocery data ---
# grocery_data = defaultdict(list)
# grocery_data['רמי לוי'].append(['גזר', 1.8])
# grocery_data['רמי לוי'].append(['בצל יבש', 2.5])
# grocery_data['שופרסל'].append(['כרוב לבן', 3.0])
# grocery_data['שופרסל'].append(['גזר', 1.9])

# --- Build message text with RTL mark ---
RTL = '\u200F'
message_lines = []
for store, items in grocery_data.items():
    message_lines.append(f"{RTL}{store}:")
    for item, price in items:
        message_lines.append(f"{RTL}- {item} ({price})")
message_text = "\n".join(message_lines)

# --- Telegram bot credentials ---
bot_token = os.getenv("DATABASE_URL")
chat_id = "-4930643833"

# --- Send message ---
url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
payload = {
    "chat_id": chat_id,
    "text": message_text
}

response = requests.post(url, data=payload)
if response.ok:
    print("Message sent successfully!")
else:
    print("Error sending message:", response.text)


Message sent successfully!
